## Problem klasyfikacji sekwencji

Dane to ciągi różnych długości, np. tekst, muzyka, film.
* Elementami ciągów są “obiekty bazowe”
  * Tekst - ciąg słów (ogólniej - tokenów)
  * Film - ciąg obrazów
  * Muzyka - ciąg dźwięków
* Uwaga 1: elementy ciągów są od siebie zależne!
* Uwaga 2: kolejność elementów jest istotna!
* Jak można pracować z ciągami?
  * Sprowadzić ciągi do reprezentacji wektorowej i użyć
  klasycznych metod uczenia maszynowego
  * Użyć metod dedykowanych do takich danych - na przykład sieci rekurencyjnych

## NLP Modele Sekwencyjne i ich zastosowania
Modelem sekwencyjnym nazwiemy model, który jako wejście otrzymuje sekwencję, ale nie musi zwracać sekwencji.

## Sieci rekurencyjne
Zasadą działania sieci rekurencyjnej jest przechowywanie wyjścia poprzedniego elementu i wykorzystania go w kolejnym kroku.

Dlaczego po prostu do sekwencji nie wykorzystać zwykłych gęstych sieci neuronowych?
* Nie są w stanie przetwarzać sekwencji o różnych długościach
* Biorą pod uwagę tylko aktualne dane wejściowe
* Nie zapamiętują informacji o poprzednich danych wejściowych

### RNN
Podstawową wersją sieci rekurencyjnej jest RNN(recurrent Neural Network)
$$ h_t = \text{tanh}(Wx_t+Uh_{t-1}+b) $$
![](Grafika/RNN-unrolled.png)

Zalety:
* Możliwość przetwarzania sekwencji o dowolnej długości
* Rozmiar modelu nie rośnie razem z długością sekwencji
* Bierze pod uwagę poprzednie stany

Wady:
* Wolne obliczenia
* Problem wykorzystywania bardzo odległych stanów
* Nie może brać pod uwagę przyszłych stanów

### Embeddingi słów
Przeanalizujmy co się dzieje w sieci, gdy
podajemy słowa w reprezentacji _one hot_:
$$ h_t = f(W^h \cdot h_{t-1}+W^x\cdot x_t +b)$$
Jeśli $x_t$ to _one-got_ z jedynką na pozycji i to 
$$W^x\cdot x_t=W^x \cdot [0,\dots, 1_i,\dots, 0]^T=W^x[:,i]$$
Zatem przekształcenie to jest równoważne wzięciu i'tej kolumny macierzy wag.

Czyli i-ta kolumna macierzy wag jest w pewnym sensie reprezentacją słowa i-tego.

Zatem pójdźmy krok dalej: stwórzmy dodatkową warstwę w sieci - macierz 
embeddingów EMB, zawierającą reprezentacje słów, które będą
przekazywane do wyliczenia stanu ukrytego
\begin{align*}
    &emb_t=EMB\cdot x_t=EMB[:,i]\\
    &h_t=f(W^h \cdot h_{t-1}+W^x\cdot emb_t +b)
\end{align*}

Embeddingi są parametrami sieci, ale
jednocześnie reprezentacją słów.
Oznacza to, że trenując sieć, uczymy

embeddingi, czyli uczymy się reprezentacji słów!

Modele sekwencyjne można podzielić na kilka przykładów
### One-to-many, wejście o długości jednostkowej, wyjście o długości > 1. 
Przykład: generacja tekstu
   
![](Grafika/rnn-one-to-many-ltr.png)

### Many-to-one
przykład klasyfikacja sentymentu

![](Grafika/rnn-many-to-one-ltr.png)

### Many-to-many (tyle samo wejść co wyjść)
przykład: NER(named entity recognition)
![](Grafika/rnn-many-to-many-same-ltr.png)

### Many-to-many
przykład tłumaczenie maszynowe

![](Grafika/rnn-many-to-many-different-ltr.png)

### Problem odległych informacji relewantnych
Rozważmy problem predykcji następnego słowa po “the clouds
are in the __”

Jest to dość proste zadanie, bo odpowiedź można łatwo
wywnioskować na podstawie tych kilku słów.

W takich przypadkach zwykła sieć RNN jest odpowiednią
strukturą.

![](Grafika/RNN-shorttermdepdencies.png)



Próba przewidzenia “[I grew up in France… . I speak fluent ___”
wymaga sięgnięcia wstecz dalej niż kilka słów.

Ostatnie słowa sugerują tylko, że następne słowo jest nazwą języka -
odgadnięcie, że chodzi o francuski, wymaga odnalezienia”France”.

W praktyce dystans do relewantnej informacji często jest duży, a w
miarę wzrostu tego dystansu, zwykła sieć RNN staje się niezdolna do
wyłapania tych zależności.

![](Grafika/RNN-longtermdependencies.png)

### Eksplodujący/Zanikający gradient
Problem ten często spotyka się podczas korzystania z RNN. Wynika to z tego, że podczas wyliczania gradientu przemnażamy przez siebie wielokrotnie gradienty dla danych chwil czasowych w związku z tym może on zacząć zanikać(jak przemnażamy małe wartości), albo "wybuchnąć"(jak przemnażamy duże wartości).

Jak sobie poradzić z tym problem?

## LSTM
Sieci Long Short Term Memory – zazwyczaj
krótko “LSTM”.
Hochreiter & Schmidhuber, 1997 (!)

Są szczególnym rodzajem sieci
rekurencyjnych zaprojektowane tak, aby
zwiększyć skuteczność wykrywania
długodystansowych zależności.

### RNN
![](Grafika/LSTM3-SimpleRNN.png)

### LSTM
![](Grafika/LSTM3-chain.png)
![](Grafika/LSTM2-notation.png)

Gdzie $\sigma$ to aktywacja sigmoid.

Zatem jak można zauważyć LSTM posiada dwie "ścieżki" pamięci. Pierwszą jest tak zwany "cell state"
![](Grafika/LSTM3-C-line.png)
Ze względu na to, że ma on tylko liniowe interakcje łatwo jest o przepływ informacji tą scieżką. LSTM ma możliwość usuwania i dodawania informacji do tej ścieżki, co jest decydowanie przez tak zwane bramki(gates). Decydują one o tym czy dana informacją powinna dalej przejść

![](Grafika/LSTM3-gate.png)

Ponieważ sigmoida zwraca wartości między 0 a 1 decyduje jak "dużo" informacji powinno przepłynąć dalej.

#### LSTM krok po kroku
W pierwszym kroku decydujemy jak wiele aktualnej informacji powinno zostać w cell state.

![](Grafika/LSTM3-focus-f.png)

Następnie decydujemy jak wiele nowej informacji powinniśmy dodać do cell state

![](Grafika/LSTM3-focus-i.png)

Dokonujemy aktualizacji cell state

![](Grafika/LSTM3-focus-C.png)

Na koniec wybieramy interesujące nas informacje z zakutalizowanego cell state, które zostaną zwrócone przez LSTM

![](Grafika/LSTM3-focus-o.png)

### Istnieją jeszcze inne warianty LSTM np. wykorzystujące cell state do bramek

![](Grafika/LSTM3-var-peepholes.png)

i takie, które wykorzystują jedną bramkę do zapominania/dodawania informacji do cell state

![](Grafika/LSTM3-var-tied.png)


### GRU a LSTM
GRU jest kolejną siecią rekurencyjną, której celem jest rozwiązanie odległych relacji między momentami czasu

![](Grafika/gru.png)

### Stacked RNN

![](Grafika/stacked_RNN.png)

### Bidirectional RNN
Czasem może nas interesować nie tylko infromacja z lewej do prawej ale także w drugą stronę, np. podczas klasyfikacji tekstu, w związku z tym aby otrzymać Biderctional RNN łączymy wyniki z dwóch sieci RNN, jedna "czyta" od lewej do prawej, a druga w drugą stronę.

In [1]:
import tensorflow as tf


In [2]:
example_input = tf.ones((1, 12, 2))

2022-07-10 10:56:53.225086: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-07-10 10:56:53.232256: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-07-10 10:56:53.232669: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-07-10 10:56:53.233379: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

In [3]:
example_input

<tf.Tensor: shape=(1, 12, 2), dtype=float32, numpy=
array([[[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]]], dtype=float32)>

In [4]:
rnn_layer = tf.keras.layers.SimpleRNN(
    units=512,
    activation="tanh",
    return_sequences=False,
    return_state=False,
)
rnn_layer(example_input).shape


TensorShape([1, 512])

In [5]:
rnn_layer = tf.keras.layers.SimpleRNN(
    units=512,
    activation="tanh",
    dropout=0.0,
    recurrent_dropout=0.0,
    return_sequences=True,
    return_state=False,
)
rnn_layer(example_input).shape


TensorShape([1, 12, 512])

In [6]:
rnn_layer = tf.keras.layers.SimpleRNN(
    units=512,
    activation="tanh",
    dropout=0.0,
    recurrent_dropout=0.0,
    return_sequences=True,
    return_state=True,
)
output, hidden_state = rnn_layer(example_input)
print(type(output))
print(output.shape, hidden_state.shape)


<class 'tensorflow.python.framework.ops.EagerTensor'>
(1, 12, 512) (1, 512)


In [7]:
bidirectional_rnn = tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(
    units=512,
    activation="tanh",
    recurrent_dropout=0.0,
    return_sequences=True,
    return_state=False,
))
output = bidirectional_rnn(example_input)
print(type(output))
print(output.shape)


<class 'tensorflow.python.framework.ops.EagerTensor'>
(1, 12, 1024)


In [8]:
embedding_layer = tf.keras.layers.Embedding(
    vocab_size,
    output_dim,
    embeddings_initializer="uniform",
    embeddings_regularizer=None,
    activity_regularizer=None,
    embeddings_constraint=None,
    mask_zero=False,
    input_length=None,
    **kwargs
)


NameError: name 'vocab_size' is not defined

Do warstwy Embedding możemy podać przetrenowane wcześniej embeddingi. Muszą one być wcześniej przygotowane jako macierz o wymiarach (vocab_size x embedding_dim). Oraz i'ty wiersz, musi odpowiadać embeddingowi tokenu, który przekształacmy na liczbę `i`.

In [9]:
import numpy as np
import tensorflow as tf

In [10]:
pretrained_embedding_matrix = np.array(range(10*30)).reshape((10,30))

In [13]:
pretrained_embedding_matrix

array([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29],
       [ 30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,
         43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59],
       [ 60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,
         73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,
         86,  87,  88,  89],
       [ 90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102,
        103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115,
        116, 117, 118, 119],
       [120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132,
        133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145,
        146, 147, 148, 149],
       [150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162,
        163, 164, 165, 166, 167, 168, 169, 170, 171,

In [12]:
pretrained_embedding = tf.keras.layers.Embedding(
    10,
    30,
    embeddings_initializer=tf.keras.initializers.Constant(pretrained_embedding_matrix),
    embeddings_regularizer=None,
    activity_regularizer=None,
    embeddings_constraint=None,
    mask_zero=False,
    input_length=None,
)



In [14]:
pretrained_embedding(np.array([[0,9]]))

<tf.Tensor: shape=(1, 2, 30), dtype=float32, numpy=
array([[[  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,
          10.,  11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,
          20.,  21.,  22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.],
        [270., 271., 272., 273., 274., 275., 276., 277., 278., 279.,
         280., 281., 282., 283., 284., 285., 286., 287., 288., 289.,
         290., 291., 292., 293., 294., 295., 296., 297., 298., 299.]]],
      dtype=float32)>

In [15]:
lstm = tf.keras.layers.LSTM(
    units=512,
    activation="tanh",
    recurrent_dropout=0.0,
    return_sequences=False,
    return_state=False,
)

In [16]:
lstm(example_input).shape

2022-07-10 11:04:22.093312: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


TensorShape([1, 512])

In [17]:
lstm = tf.keras.layers.LSTM(
    units=512,
    activation="tanh",
    recurrent_dropout=0.0,
    return_sequences=True,
    return_state=False,
)
lstm(example_input).shape

TensorShape([1, 12, 512])

In [18]:
lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
    units=512,
    activation="tanh",
    recurrent_dropout=0.0,
    return_sequences=True,
    return_state=False,
))
lstm(example_input).shape

TensorShape([1, 12, 1024])

In [19]:
lstm = tf.keras.layers.LSTM(
    units=512,
    activation="tanh",
    recurrent_dropout=0.0,
    return_sequences=True,
    return_state=True,
)
out, cell_state, hidden_state = lstm(example_input)
out.shape, cell_state.shape, hidden_state.shape

(TensorShape([1, 12, 512]), TensorShape([1, 512]), TensorShape([1, 512]))

In [20]:
lstm(example_input, initial_state=[cell_state, hidden_state])

[<tf.Tensor: shape=(1, 12, 512), dtype=float32, numpy=
 array([[[ 9.8077089e-02, -4.1555081e-02, -2.3201019e-02, ...,
          -2.7695877e-02,  3.2766566e-02, -3.3441110e-04],
         [ 9.8540857e-02, -4.2712249e-02, -2.3906732e-02, ...,
          -2.7826820e-02,  3.2997672e-02,  9.2046765e-05],
         [ 9.8867767e-02, -4.3699343e-02, -2.4513017e-02, ...,
          -2.7938776e-02,  3.3198118e-02,  4.1465153e-04],
         ...,
         [ 9.9724382e-02, -4.7378272e-02, -2.6810193e-02, ...,
          -2.8409336e-02,  3.3902276e-02,  1.0597191e-03],
         [ 9.9784233e-02, -4.7606535e-02, -2.6954010e-02, ...,
          -2.8445369e-02,  3.3928175e-02,  1.0441197e-03],
         [ 9.9840038e-02, -4.7790918e-02, -2.7069727e-02, ...,
          -2.8475612e-02,  3.3942550e-02,  1.0199380e-03]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 512), dtype=float32, numpy=
 array([[ 9.98400375e-02, -4.77909185e-02, -2.70697270e-02,
         -3.12656015e-02,  7.51215443e-02,  1.62024964e-02,
         

In [21]:
rnn_layer = tf.keras.layers.GRU(
    units=512,
    activation="tanh",
    return_sequences=False,
    return_state=False,
)
rnn_layer(example_input).shape


TensorShape([1, 512])

In [22]:
rnn_layer = tf.keras.layers.GRU(
    units=512,
    activation="tanh",
    dropout=0.0,
    recurrent_dropout=0.0,
    return_sequences=True,
    return_state=False,
)
rnn_layer(example_input).shape


TensorShape([1, 12, 512])

In [23]:
rnn_layer = tf.keras.layers.GRU(
    units=512,
    activation="tanh",
    dropout=0.0,
    recurrent_dropout=0.0,
    return_sequences=True,
    return_state=True,
)
output, hidden_state = rnn_layer(example_input)
print(type(output))
print(output.shape, hidden_state.shape)


<class 'tensorflow.python.framework.ops.EagerTensor'>
(1, 12, 512) (1, 512)


In [24]:
bidirectional_rnn = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(
    units=512,
    activation="tanh",
    recurrent_dropout=0.0,
    return_sequences=True,
    return_state=False,
))
output = bidirectional_rnn(example_input)
print(type(output))
print(output.shape)


<class 'tensorflow.python.framework.ops.EagerTensor'>
(1, 12, 1024)


In [25]:
rnn_layer1 = tf.keras.layers.GRU(
    units=512,
    activation="tanh",
    dropout=0.0,
    recurrent_dropout=0.0,
    return_sequences=True,
    return_state=False,
)
rnn_layer2 = tf.keras.layers.GRU(
    units=512,
    activation="tanh",
    dropout=0.0,
    recurrent_dropout=0.0,
    return_sequences=False,
    return_state=False,
)
mod = tf.keras.Sequential()
mod.add(rnn_layer1)
mod.add(rnn_layer2)

In [27]:
mod.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_4 (GRU)                 (1, 12, 512)              792576    
                                                                 
 gru_5 (GRU)                 (1, 512)                  1575936   
                                                                 
Total params: 2,368,512
Trainable params: 2,368,512
Non-trainable params: 0
_________________________________________________________________


In [28]:
mod(example_input).shape

TensorShape([1, 512])

In [29]:
rnn_layer1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(
    units=512,
    activation="tanh",
    dropout=0.0,
    recurrent_dropout=0.0,
    return_sequences=True,
    return_state=False,
))
rnn_layer2 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(
    units=512,
    activation="tanh",
    dropout=0.0,
    recurrent_dropout=0.0,
    return_sequences=False,
    return_state=False,
))
mod = tf.keras.Sequential()
mod.add(rnn_layer1)
mod.add(rnn_layer2)

In [30]:
mod(example_input).shape

TensorShape([1, 1024])

### Case study: IMBD
Klasyfikacja tekstu przy użyciu RNN

In [37]:
from keras import utils

from keras.datasets import imdb


In [58]:
max_features = 1000
maxlen = 50
batch_size = 32
embedding_dims = 50
epochs = 3


In [59]:
import numpy as np


(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)


In [60]:
# (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

print(len(x_train), "train sequences")
print(len(x_test), "test sequences")
print(x_train[:3])


25000 train sequences
25000 test sequences
[list([1, 14, 22, 16, 43, 530, 973, 2, 2, 65, 458, 2, 66, 2, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 2, 2, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2, 19, 14, 22, 4, 2, 2, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 2, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2, 2, 16, 480, 66, 2, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 2, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 2, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 2, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 2, 88, 12, 16, 283, 5, 16, 2, 113, 103, 32, 15, 16, 2, 19, 178, 32])
 list([1, 194, 2, 194, 2, 78, 228, 5, 6, 2, 2, 2, 134, 26, 4, 715, 8, 118, 2, 1

Zwróćmy uwagę w powyższym, że ciągi zaczynają się zawsze od "1" - jest to oznaczenie początku zdania. Czyli "początek zdania" będzie miał swój embedding.

Standaryzacja długości sekwencji (Padding)

In [61]:
import keras

In [62]:
keras.__version__

'2.9.0'

In [63]:
# from keras import sequence
# keras < 2.9

# x_train = sequence.pad_sequences(x_train, maxlen=maxlen, padding="pre")
# x_test = sequence.pad_sequences(x_test, maxlen=maxlen, padding="pre")

# print("x_train shape:", x_train.shape)
# print("x_test shape:", x_test.shape)
# print(x_train[0])

In [64]:
from keras import utils


x_train = utils.pad_sequences(x_train, maxlen=maxlen, padding="pre")
x_test = utils.pad_sequences(x_test, maxlen=maxlen, padding="pre")

print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)
print(x_train[0])


x_train shape: (25000, 50)
x_test shape: (25000, 50)
[  2  56  26 141   6 194   2  18   4 226  22  21 134 476  26 480   5 144
  30   2  18  51  36  28 224  92  25 104   4 226  65  16  38   2  88  12
  16 283   5  16   2 113 103  32  15  16   2  19 178  32]


In [65]:
n_train = 3000
n_test = 500
x_train = x_train[:n_train]
y_train = y_train[:n_train]
x_test = x_test[:n_test]
y_test = y_test[:n_test]


In [66]:
y_train.shape


(3000,)

In [67]:
y_train

array([1, 0, 0, ..., 1, 0, 0])

### Zadania

### Zwykła sieć rekurencyjna ( z embeddingami)

In [1]:
from keras.models import Sequential
from keras.layers import (
    Dense,
    Dropout,
    Embedding,
    SimpleRNN,
    LSTM,
    Bidirectional,
    GRU
)

from keras.callbacks import EarlyStopping


### Simple RNN + dense pomiędzy zwracanym wyjściem z RNN a outputem

In [73]:
model = Sequential()

model.add(Embedding(max_features, 50))
model.add(SimpleRNN(64))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


model.summary()

early = EarlyStopping(patience=2)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 50)          50000     
                                                                 
 simple_rnn_7 (SimpleRNN)    (None, 64)                7360      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 57,425
Trainable params: 57,425
Non-trainable params: 0
_________________________________________________________________


In [74]:
model.fit(x_train, y_train, callbacks=[early], validation_split=0.1, batch_size=64, epochs=5)

Epoch 1/5
43/43 [==============================] - 11s 230ms/step - loss: 0.6938 - accuracy: 0.5059 - val_loss: 0.6886 - val_accuracy: 0.5200
Epoch 2/5
43/43 [==============================] - 10s 234ms/step - loss: 0.6396 - accuracy: 0.6667 - val_loss: 0.7422 - val_accuracy: 0.5033
Epoch 3/5
43/43 [==============================] - 10s 230ms/step - loss: 0.4991 - accuracy: 0.7756 - val_loss: 0.5757 - val_accuracy: 0.6733
Epoch 4/5
43/43 [==============================] - 10s 233ms/step - loss: 0.3062 - accuracy: 0.8948 - val_loss: 0.6378 - val_accuracy: 0.6767
Epoch 5/5
43/43 [==============================] - 10s 225ms/step - loss: 0.1749 - accuracy: 0.9530 - val_loss: 0.6833 - val_accuracy: 0.7000


## Dwukierunkowa sieć rekurencyjna

In [75]:
model = Sequential()
model.add(Embedding(max_features, embedding_dims, input_length=maxlen, mask_zero=True))

model.add(Bidirectional(SimpleRNN(32)))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation="sigmoid"))

model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
)
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 50, 50)            50000     
                                                                 
 bidirectional_5 (Bidirectio  (None, 64)               5312      
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 16)                1040      
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 56,369
Trainable params: 56,369
Non-trainable params: 0
_________________________________________________________________


In [76]:
model.fit(x_train, y_train, callbacks=[early], validation_split=0.1, batch_size=64, epochs=5)

Epoch 1/5
43/43 [==============================] - 31s 689ms/step - loss: 0.6952 - accuracy: 0.5170 - val_loss: 0.6854 - val_accuracy: 0.5567
Epoch 2/5
 4/43 [=>............................] - ETA: 24s - loss: 0.6685 - accuracy: 0.6992

KeyboardInterrupt: 

### Zadanie. Powtórz powyższe modele z komórką LSTM

In [ ]:
from tensorflow import random
random.set_seed(2)

model = Sequential()
model.add(Embedding(
    max_features,
    400))
model.add(Bidirectional(LSTM(
    units=embedding_dims,
    activation="tanh",
    return_sequences=False,
    return_state=False,
)))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss='binary_crossentropy', optimizer='adam',
              metrics=['accuracy'])

model.summary()

early = EarlyStopping(patience=2)

model.fit(x_train, y_train, callbacks=[early], validation_split=0.1, batch_size=64, epochs=5)

model.evaluate(x_test, y_test, verbose=0)


In [77]:
model = Sequential()
model.add(Embedding(max_features, embedding_dims, input_length=maxlen, mask_zero=True))

model.add(Bidirectional(LSTM(32)))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation="sigmoid"))

model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
)
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 50, 50)            50000     
                                                                 
 bidirectional_6 (Bidirectio  (None, 64)               21248     
 nal)                                                            
                                                                 
 dense_6 (Dense)             (None, 16)                1040      
                                                                 
 dense_7 (Dense)             (None, 1)                 17        
                                                                 
Total params: 72,305
Trainable params: 72,305
Non-trainable params: 0
_________________________________________________________________


In [78]:
model.fit(x_train, y_train, callbacks=[early], validation_split=0.1, batch_size=64, epochs=5)

Epoch 1/5


2022-07-10 12:47:54.369529: W tensorflow/core/common_runtime/forward_type_inference.cc:231] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_LEGACY_VARIANT
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_FLOAT
    }
  }
}

	while inferring type of node 'cond_40/output/_19'


16/43 [==========>...................] - ETA: 29s - loss: 0.6929 - accuracy: 0.4971

KeyboardInterrupt: 

## Dwuwarstwowa sieć rekurencyjna

In [81]:
model = Sequential()
model.add(Embedding(max_features, embedding_dims, input_length=maxlen, mask_zero=True))

model.add(Bidirectional(GRU(32, return_sequences=True)))
model.add(Bidirectional(GRU(32, return_sequences=True)))
model.add(Bidirectional(GRU(64)))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation="sigmoid"))

model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
)
model.summary()

model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=100,
    callbacks=[early],
    validation_split=0.10,
)

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 50, 50)            50000     
                                                                 
 bidirectional_13 (Bidirecti  (None, 50, 64)           16128     
 onal)                                                           
                                                                 
 bidirectional_14 (Bidirecti  (None, 50, 64)           18816     
 onal)                                                           
                                                                 
 bidirectional_15 (Bidirecti  (None, 128)              49920     
 onal)                                                           
                                                                 
 dense_10 (Dense)            (None, 16)                2064      
                                                     

KeyboardInterrupt: 

### CNN a tekst

Mając sekwencję możemy zamiast wykorzystywać RNN skorzystać z jednowymiarowej konwolucji. W tym przypadku rozmiar kernela(jądra) decyduje o tym na ile chwil czasowych jednocześnie patrzy CNN. 

In [16]:
from keras.layers import Conv1D, GlobalAveragePooling1D, GlobalAveragePooling2D

In [83]:
GlobalAveragePooling2D()(tf.zeros((16, 32,32,3))).shape

TensorShape([16, 3])

#### Zadanie
korzystając z warstwy `tf.keras.layers.Conv1D` i jakiejś warstwy poolingowej zastąpić w poprzednim modelu RNN siecią CNN.
Argumenty są takie same jak dla konwolucji 2D tylko `kernel_size` i `strides` mogą być tylko liczbami całkowitymi.

In [84]:
from keras.layers import Conv1D, GlobalAveragePooling1D, GlobalAveragePooling2D

model = Sequential()
model.add(Embedding(max_features, 64, input_length=maxlen))

model.add(Conv1D(32, kernel_size=5))
model.add(Conv1D(64, kernel_size=5))
model.add(Conv1D(128, kernel_size=5))
model.add(GlobalAveragePooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation="sigmoid"))

model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
)
model.summary()

model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=100,
    callbacks=[early],
    validation_split=0.10,
)

In [89]:
model.evaluate(x_test, y_test)

16/16 [==============================] - 0s 27ms/step - loss: 0.5996 - accuracy: 0.7380


[0.5995662212371826, 0.7379999756813049]

### Case Study
Utwórz model, który identyfikuje emocje wpisu na twiterze.

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('Dane/tweet_emotions.csv')

In [4]:
data.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [5]:
X = data['content'].tolist()
y = data['sentiment']

### Zadanie 5
Przygotowanie danych:
* (Opcjonalnie) Przeczyścić
* Tokenizować i utworzyć słownik
* Zakodowanie etykiet
* Podział na zbiór treningowy i testowy

`tf.keras.layers.TextVectorization`

metody:

`adapt`

In [9]:
import tensorflow as tf
import numpy as np

In [10]:
def preprocess_tweets(sent):
    outsent = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",sent).split())
    outsent = outsent.lower()
    return outsent

import re

X = [preprocess_tweets(elem) for elem in X]


# Tworzymy warstwe tokenizującą
vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=5000, output_mode="int"
)
# Dopasujemy tokenizator do danych
vectorize_layer.adapt(X) #tworzy słownik


from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_label = le.fit_transform(y)

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(np.array(X), y_label, test_size=0.1)

In [37]:
x_train.shape

(36000,)

In [11]:
y_train

array([ 5,  5, 12, ...,  4, 11, 12])

In [12]:
len(le.classes_)

13

### Zadanie 6
przetrenować sieć CNN do klasyfikacji sentymentu

In [25]:
#Tokenizacja
vectorize_layer = tf.keras.layers.TextVectorization(max_tokens=5000, output_mode="int")
vectorize_layer.adapt(X)
vocab_size = len(vectorize_layer.get_vocabulary())

In [26]:
from gensim import downloader
import numpy as np

glove_vectors = downloader.load('glove-wiki-gigaword-100')

embedding_matrix = np.zeros((vocab_size, 100))

for i, word in enumerate(vectorize_layer.get_vocabulary()):
    try:
        word_embedding = glove_vectors[word]
        embedding_matrix[i] = word_embedding
    except KeyError:
        pass



In [28]:


#Zakodowanie etykiet
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
enc.fit(y)
list(enc.classes_)

#split 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(np.array(X), enc.transform(y), test_size=0.1)

#model
model = Sequential()
model.add(vectorize_layer)
model.add(Embedding(vocab_size, 100, 
                    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix)
                   ))
model.add(Conv1D(32, kernel_size=5, padding='same'))
model.add(Conv1D(64, kernel_size=5, padding='same'))
model.add(GlobalAveragePooling1D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(len(enc.classes_), activation="softmax"))

# model.layers[1].trainable=False
model.compile(
    loss=tf.keras.losses.sparse_categorical_crossentropy, optimizer=tf.keras.optimizers.Adam(0.0001), metrics=["sparse_categorical_accuracy"]
)
model.summary()

model.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=100,
#     callbacks=EarlyStopping(patience=2),
    validation_split=0.1
)


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_3 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding_8 (Embedding)     (None, None, 100)         500000    
                                                                 
 conv1d_20 (Conv1D)          (None, None, 32)          16032     
                                                                 
 conv1d_21 (Conv1D)          (None, None, 64)          10304     
                                                                 
 global_average_pooling1d_6   (None, 64)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_20 (Dense)            (None, 256)              

254/254 [==============================] - 6s 25ms/step - loss: 1.7560 - sparse_categorical_accuracy: 0.4035 - val_loss: 1.9099 - val_sparse_categorical_accuracy: 0.3439
Epoch 36/100
254/254 [==============================] - 6s 25ms/step - loss: 1.7508 - sparse_categorical_accuracy: 0.4053 - val_loss: 1.9021 - val_sparse_categorical_accuracy: 0.3439
Epoch 37/100
254/254 [==============================] - 6s 25ms/step - loss: 1.7456 - sparse_categorical_accuracy: 0.4070 - val_loss: 1.9173 - val_sparse_categorical_accuracy: 0.3486
Epoch 38/100
254/254 [==============================] - 6s 26ms/step - loss: 1.7433 - sparse_categorical_accuracy: 0.4094 - val_loss: 1.9087 - val_sparse_categorical_accuracy: 0.3433
Epoch 39/100
254/254 [==============================] - 7s 26ms/step - loss: 1.7354 - sparse_categorical_accuracy: 0.4097 - val_loss: 1.9167 - val_sparse_categorical_accuracy: 0.3508
Epoch 40/100
254/254 [==============================] - 6s 25ms/step - loss: 1.7312 - sparse_categ

KeyboardInterrupt: 

In [ ]:
model.evaluate(X_test, y_test)

### Zadanie 7
Przetrenować RNN do klasyfikacji sentymentu

In [31]:
rnn_layer1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(
    units=512,
    activation="tanh",
    dropout=0.0,
    recurrent_dropout=0.0,
    return_sequences=True,
    return_state=False,
))
rnn_layer2 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(
    units=512,
    activation="tanh",
    dropout=0.0,
    recurrent_dropout=0.0,
    return_sequences=False,
    return_state=False,
))

In [35]:
model = Sequential()
model.add(vectorize_layer)
model.add(Embedding(vocab_size, 64))

model.add(rnn_layer1)
model.add(rnn_layer2)
model.add(Dense(64, activation='relu'))
model.add(Dense(13, activation="sigmoid"))

model.compile(
    loss=tf.keras.losses.sparse_categorical_crossentropy, optimizer="adam", metrics=["sparse_categorical_accuracy"]
)
model.summary()

model.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=5,
    callbacks=EarlyStopping(patience=2),
    validation_split=0.10,
)

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_3 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding_12 (Embedding)    (None, None, 64)          320000    
                                                                 
 bidirectional_6 (Bidirectio  (None, None, 1024)       1775616   
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 1024)             4724736   
 nal)                                                            
                                                                 
 dense_29 (Dense)            (None, 64)                65600     
                                                     

In [36]:
model.evaluate(x_test, y_test)

125/125 [==============================] - 3s 21ms/step - loss: 2.1852 - sparse_categorical_accuracy: 0.2020


[2.1851632595062256, 0.20200000703334808]

In [29]:
model = Sequential()
model.add(vectorize_layer)
model.add(Embedding(vocab_size, 100, embeddings_initializer=tf.keras.initializers.Constant(
            embedding_matrix
        )))

model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(Bidirectional(LSTM(256)))
model.add(Dense(256, activation='relu'))
model.add(Dense(len(enc.classes_), activation="softmax"))
# model.layers[1].trainable=False
model.compile(
    loss=tf.keras.losses.sparse_categorical_crossentropy, optimizer=tf.keras.optimizers.Adam(0.001), metrics=["sparse_categorical_accuracy"]
)
model.summary()

model.fit(
    X_train,
    y_train,
    batch_size=128,
    epochs=100,
    callbacks=EarlyStopping(patience=2),
    validation_split=0.1
)


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_3 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding_9 (Embedding)     (None, None, 100)         500000    
                                                                 
 bidirectional_3 (Bidirectio  (None, None, 512)        731136    
 nal)                                                            
                                                                 
 bidirectional_4 (Bidirectio  (None, None, 512)        1574912   
 nal)                                                            
                                                                 
 bidirectional_5 (Bidirectio  (None, 512)              1574912   
 nal)                                                 

In [30]:
model.evaluate(X_test, y_test)

125/125 [==============================] - 4s 29ms/step - loss: 1.8774 - sparse_categorical_accuracy: 0.3505


[1.8774006366729736, 0.3504999876022339]

#### Contextual embeddings
Przy użyciu sieci rekurencyjnych można tworzyć embeddingi, które uwzględniają kontekst w jakim użyte jest dane słowo. Istnieje wiele słów, które jak są wykorzystane bez kontekstu nie można jednoznacznie określić ich znaczenia(np. zamek jako budowla i jako zapięcie kurtki). W związku z tym wykorzystuje się modele sekwencyjne, które modelują słowo w zależności od jego "otoczenia"
##### ELMo
Model ELMo wykorzsytuje sieci Bidirectional RNN do reprezentacji kontekstu

![](Grafika/Bert-language-modeling.png)

Działanie ELMo

![](Grafika/elmo-forward-backward-language-model-embedding.png)

![](Grafika/elmo-embedding.png)

In [38]:
import tensorflow_hub as hub


In [39]:
elmo = hub.load("https://tfhub.dev/google/elmo/3").signatures["default"]


In [40]:
x = ["ELMo lives on sesame street."]

# Extract ELMo features
embeddings = elmo(tf.constant(x))["elmo"]

embeddings.shape


TensorShape([1, 5, 1024])

In [51]:
X_elmo_embeddings = elmo(tf.constant(x_train[:100]))["elmo"]

In [49]:
X_elmo_embeddings.shape

TensorShape([100, 28, 1024])

In [43]:
model_elmo = Sequential()
model_elmo.add(Bidirectional(GRU(512)))
model_elmo.add(Dense(512, activation="relu"))
model_elmo.add(Dropout(0.4))
model_elmo.add(Dense(13, activation="softmax"))

In [44]:
model_elmo.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer="adam",
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

In [47]:
model_elmo.fit(
    X_elmo_embeddings,
    y_train[:100],
    batch_size=16,
    epochs=2,
    validation_split=0.10,
)

Epoch 1/2
6/6 [==============================] - 4s 156ms/step - loss: 2.5344 - sparse_categorical_accuracy: 0.1889 - val_loss: 2.4384 - val_sparse_categorical_accuracy: 0.3000
Epoch 2/2
6/6 [==============================] - 0s 38ms/step - loss: 1.6741 - sparse_categorical_accuracy: 0.4667 - val_loss: 2.3752 - val_sparse_categorical_accuracy: 0.3000


### Seq2Seq
Modele Seq2Seq składają się z dwóch modeli sekwencyjnych, zazwyczaj pierwszy nazywa się `encoder` a drugi `decoder`. Zazwyczaj sekwencje wejściowe i wyjściowe mogą być różnej długości.
Przykłady zastosowań
* Machine translation
* Table summarization
* Image captioning
* Document Summarization
* Question Answering(np. chatboty)
* Speech recognition

![](Grafika/seq2seq-teacher-forcing.png)

Aby utworzy takie modele trzeba już korzystać z functional API w Kerasie, ponieważ tutaj nie mamy prostego liniowego potoku zmiennych.

In [52]:
import keras

In [54]:
img_input = keras.Input(shape=(32,32,3))

flatten_layer = keras.layers.Flatten()

flattened = flatten_layer(img_input)
x = keras.layers.Dense(256, activation='relu')(flattened)
x1 = keras.layers.Dense(256, activation='relu')(x)
x1 = x + x1
output = keras.layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs=img_input, outputs=output)

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 flatten_1 (Flatten)         (None, 3072)              0         
                                                                 
 dense_35 (Dense)            (None, 256)               786688    
                                                                 
 dense_37 (Dense)            (None, 10)                2570      
                                                                 
Total params: 789,258
Trainable params: 789,258
Non-trainable params: 0
_________________________________________________________________


### Spacy i problemy lingwistyczne
spacy jest biblioteką zawierającą wiele modeli do problemów lingwistycznych, które teraz sobie krótko omówimy.

##### tokenizacja
spacy także posiada tokenizacje

In [55]:
import spacy


In [56]:
!python -m spacy download pl_core_news_sm

/bin/bash: /home/mchraba/miniconda3/envs/szkolenie_ds/lib/libtinfo.so.6: no version information available (required by /bin/bash)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 13.0 MB/s eta 0:00:00m eta 0:00:010:01:01
✔ Download and installation successful
You can now load the package via spacy.load('pl_core_news_sm')


In [58]:
# wczytywanie modelu językowego
nlp = spacy.load("pl_core_news_sm")
doc = nlp("biorę dzisiaj udział w zajęciach z sieci rekurencyjnych.")
for token in doc:
    print(token.text)
    print(token.lemma_)


biorę
biora
dzisiaj
dzisiaj
udział
udział
w
w
zajęciach
zajęcia
z
z
sieci
sieć
rekurencyjnych
rekurencyjny
.
.


#### Part-Of_Speech (POS) - tagging
problem ten polega na wyjaśnieniu w jaki sposób dane słowo jest wykorzystane w zdaniu. Ustalone jest 8 części mowy (po ang bo będzie łatwiej):
* Noun
* Pronoun
* Adjective
* Verb
* Adverb
* Preposition
* Conjunction
* Interjection

In [59]:
doc = nlp("uczymy się teraz o Spacy")

# Iterate over the tokens
for token in doc:
    # Print the token and its part-of-speech tag
    print(token, token.tag_, token.pos_, spacy.explain(token.tag_))


uczymy FIN VERB None
się QUB PRON None
teraz ADV ADV adverb
o PREP ADP None
Spacy SUBST PROPN None


/home/mchraba/miniconda3/envs/szkolenie_ds/lib/python3.9/site-packages/spacy/glossary.py:19: UserWarning: [W118] Term 'FIN' not found in glossary. It may however be explained in documentation for the corpora used to train the language. Please check `nlp.meta["sources"]` for any relevant links.
  warnings.warn(Warnings.W118.format(term=term))
/home/mchraba/miniconda3/envs/szkolenie_ds/lib/python3.9/site-packages/spacy/glossary.py:19: UserWarning: [W118] Term 'QUB' not found in glossary. It may however be explained in documentation for the corpora used to train the language. Please check `nlp.meta["sources"]` for any relevant links.
  warnings.warn(Warnings.W118.format(term=term))
/home/mchraba/miniconda3/envs/szkolenie_ds/lib/python3.9/site-packages/spacy/glossary.py:19: UserWarning: [W118] Term 'PREP' not found in glossary. It may however be explained in documentation for the corpora used to train the language. Please check `nlp.meta["sources"]` for any relevant links.
  warnings.warn(

In [60]:
doc = nlp("uczymy się teraz o Spacy")
spacy.displacy.render(doc, style="dep", jupyter=True)

#### Dependency Parsing
Jest to proces tworzenia struktury gramatycznej zdania. Daje on nam zależność słów w zdaniu.

In [65]:
doc = nlp("uczymy się teraz o Spacy")

for token in doc:
    print(token.text, "-->", token.dep_)


uczymy --> ROOT
się --> amod
teraz --> dobj
o --> dobj
Spacy --> dobj


In [66]:
token.vector

array([-0.41865864,  0.43066335,  0.6623521 , -0.20936172,  0.48971444,
        0.49390063,  0.64525855, -0.90870166,  0.09301043,  0.18278256,
       -0.74089634, -0.05980694,  0.22271365,  0.34179288, -0.5375267 ,
        0.7238251 , -0.9606206 ,  0.12817085, -0.56941235,  1.1561015 ,
       -0.09720856, -0.50086236, -0.75893176,  0.42354727,  0.10997573,
        0.7482457 , -1.2016877 ,  0.8505768 ,  0.3301307 , -0.9260626 ,
       -0.39288688,  0.61222076, -0.94754726, -0.41945955, -0.36880997,
        0.00419371,  0.14100781, -0.771331  ,  0.17537194, -0.1911444 ,
        0.28079316, -0.37717414,  0.39071086, -0.782937  , -0.23812774,
        0.10956369,  0.11842769, -0.40983358, -0.0601393 ,  1.0633551 ,
        0.96898174, -0.29064146, -0.86304533, -0.57538044,  0.45362335,
       -0.7393721 , -0.89779377,  0.5219121 , -0.6684075 , -0.5712844 ,
       -0.71596515,  0.10056004, -0.10889052,  0.349348  ,  0.26711488,
       -0.20262252, -0.02398084,  1.1425614 , -0.6714214 , -0.75

#### Named Entity Recogniton
Czyli po prostu wykrywanie nazw własnych

In [62]:
# nlp = spacy.load("en_core_web_sm")
doc = nlp(
    "W Warszawie ostatnio otworzono nowe stacje drugiej lini metra."
)
# See the entity present
print(f"Enitites: {doc.ents}")
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)


Enitites: (Warszawie, lini metra)
Warszawie 2 11 placeName
lini metra 51 61 persName


#### Entity Recogniton
nazywane też Entity Detection jest bardziej zaawansowany od NER, ponieważ rozpoznaje istotne elementy między innymi miejsca, ludzi, organizacje, języki itp.

In [63]:
nlp = spacy.load("en_core_web_sm")
doc = nlp(
    """The Amazon rainforest,[a] alternatively, the Amazon Jungle, also known in English as Amazonia, is a moist broadleaf tropical rainforest in the Amazon biome that covers most of the Amazon basin of South America. This basin encompasses 7,000,000 km2 (2,700,000 sq mi), of which 5,500,000 km2 (2,100,000 sq mi) are covered by the rainforest. This region includes territory belonging to nine nations.

The majority of the forest is contained within Brazil, with 60% of the rainforest, followed by Peru with 13%, Colombia with 10%, and with minor amounts in Bolivia, Ecuador, French Guiana, Guyana, Suriname, and Venezuela. Four nations have "Amazonas" as the name of one of their first-level administrative regions and France uses the name "Guiana Amazonian Park" for its rainforest protected area. The Amazon represents over half of the planet's remaining rainforests,[2] and comprises the largest and most biodiverse tract of tropical rainforest in the world, with an estimated 390 billion individual trees divided into 16,000 species.[3]

Etymology
The name Amazon is said to arise from a war Francisco de Orellana fought with the Tapuyas and other tribes. The women of the tribe fought alongside the men, as was their custom.[4] Orellana derived the name Amazonas from the Amazons of Greek mythology, described by Herodotus and Diodorus.[4]

History
See also: History of South America § Amazon, and Amazon River § History
Tribal societies are well capable of escalation to all-out wars between tribes. Thus, in the Amazonas, there was perpetual animosity between the neighboring tribes of the Jivaro. Several tribes of the Jivaroan group, including the Shuar, practised headhunting for trophies and headshrinking.[5] The accounts of missionaries to the area in the borderlands between Brazil and Venezuela have recounted constant infighting in the Yanomami tribes. More than a third of the Yanomamo males, on average, died from warfare.[6]"""
)

entities = [(i, i.label_, i.label) for i in doc.ents]
entities


[(Amazon, 'ORG', 383),
 (the Amazon Jungle, 'GPE', 384),
 (English, 'LANGUAGE', 389),
 (Amazonia, 'GPE', 384),
 (Amazon, 'ORG', 383),
 (Amazon, 'ORG', 383),
 (South America, 'LOC', 385),
 (7,000,000, 'CARDINAL', 397),
 (2,700,000, 'CARDINAL', 397),
 (5,500,000, 'CARDINAL', 397),
 (2,100,000, 'CARDINAL', 397),
 (nine, 'CARDINAL', 397),
 (Brazil, 'GPE', 384),
 (60%, 'PERCENT', 393),
 (Peru, 'GPE', 384),
 (13%, 'PERCENT', 393),
 (Colombia, 'GPE', 384),
 (10%, 'PERCENT', 393),
 (Bolivia, 'GPE', 384),
 (Ecuador, 'GPE', 384),
 (French Guiana, 'PERSON', 380),
 (Guyana, 'GPE', 384),
 (Suriname, 'GPE', 384),
 (Venezuela, 'GPE', 384),
 (Four, 'CARDINAL', 397),
 (Amazonas, 'ORG', 383),
 (one, 'CARDINAL', 397),
 (first, 'ORDINAL', 396),
 (France, 'GPE', 384),
 (Guiana Amazonian Park, 'WORK_OF_ART', 388),
 (Amazon, 'ORG', 383),
 (over half, 'CARDINAL', 397),
 (an estimated 390 billion, 'MONEY', 394),
 (16,000, 'CARDINAL', 397),
 (Amazon, 'ORG', 383),
 (Francisco de Orellana, 'ORG', 383),
 (Tapuyas,

In [64]:
spacy.displacy.render(doc, style="ent", jupyter=True)


#### Transformers
Transformery wywołały ogromny przełom w NLP, są one w stanie modelować relacje między dowolnie odległymi chwilami czasu i są szybsze ponieważ nie wymagają pętli(wystarcza tylko mnożenie wektorów i macierzy)

![](Grafika/transformer.png)

Czym jest `Positional Encoding`? Istnieją dwie szkoły tworzenia
* Uczymy embeddingi pozycji razem z modelem, czyli `Positional Encoding` jest parameterem modelu o ustalonej długości
* Wykorzystujemy z góry zdefiniowaną funkcję np.
\begin{align*}
  p_{i,j}=
  \begin{cases}
    \sin\left(\frac{i}{10000^{j/d}}\right)\\
    \cos\left(\frac{i}{10000^{(j-1)/d}}\right)
  \end{cases}
\end{align*}
gdzie $i$-chwila czasu, $j$-j'ty wymiar w wektorze positional encodingu, $d$-wymiar positional encodingu.

Positional Encoding jest potrzebny, żeby model był w stanie modelować dane wejściowe jako sekwencje, w przeciwnym wypadku, pozycja w której umieścimy token/wartość w chwili czasu nie ma żadnego znaczenia.

Nakładamy maskę, żeby wyliczać atencję tylko na podstawie tokenów/chwil czasu, które chcemy, żeby model brał pod uwagę. Np. podczas uczenia generatora tekstu, będziemy maskowali wszystkie następne tokeny, ponieważ nie chcemy, żeby model genrował token na podstawie przyszłych tokenów, natomiast w przypadku klasyfikacji tekstu już taka maska nie jest potrzebna. Podczas trenowania nakłada też się zawsze maskę na tokeny odpowiadające paddingowi.
  

In [67]:
from transformers import BertTokenizer, TFBertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained("bert-base-uncased")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='tf')
output = model(encoded_input)

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [70]:
encoded_input

{'input_ids': <tf.Tensor: shape=(1, 12), dtype=int32, numpy=
array([[ 101, 5672, 2033, 2011, 2151, 3793, 2017, 1005, 1040, 2066, 1012,
         102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 12), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 12), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [69]:
output.last_hidden_state.shape

TensorShape([1, 12, 768])

### Bibliografia
#### Sieci rekurencyjne
* [RNN i LSTM](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)
* [ELMo-Embeddingi z kontekstem](https://arxiv.org/abs/1802.05365v2)
* [BERT ELMo zilustrowane](https://jalammar.github.io/illustrated-bert/)
* [Bahdanau Attention](https://arxiv.org/abs/1508.04025)
#### Transformery
* [Artykuł wprowadzający transformery](https://arxiv.org/abs/1706.03762)
* [Wizualizacje działania transformerów](https://jalammar.github.io/illustrated-transformer/)
* [BERT](https://arxiv.org/abs/1810.04805)

## Tematy
### Sub-word tokenizers:
* [Byte-Pair Encoding](https://arxiv.org/abs/1508.07909)
* [WordPiece](https://ai.googleblog.com/2021/12/a-fast-wordpiece-tokenization-system.html)
* [Unigram Language Model](https://arxiv.org/pdf/1804.10959.pdf)
* [SentencePiece](https://jacky2wong.medium.com/understanding-sentencepiece-under-standing-sentence-piece-ac8da59f6b08)
### Embeddingi
* [Word2Vec](https://arxiv.org/abs/1301.3781)
* [GloVe](https://nlp.stanford.edu/projects/glove/)
* [FastText](https://arxiv.org/pdf/1607.04606.pdf)
* [Doc2Vec](https://arxiv.org/pdf/1301.3781.pdf)
## Biblioteki:
* [NLTK](https://www.nltk.org/)
* [Spacy](https://spacy.io/)
* [FastText](https://fasttext.cc/)
* [Hugging face tokenizers](https://huggingface.co/docs/tokenizers/python/latest/)
* [Gensim](https://radimrehurek.com/gensim/)